In [1]:
import pandas as pd

df = pd.read_csv('safety_recall_data.csv')
df2 = pd.read_csv('vehicle_info.csv')

In [2]:
df.head

<bound method NDFrame.head of      Unnamed: 0         Manufacturer NHTSACampaignNumber parkIt parkOutSide  \
0             0   Ford Motor Company           22V411000  False       False   
1             1   Ford Motor Company           22V411000  False       False   
2             2   Ford Motor Company           22V451000  False       False   
3             3   Ford Motor Company           22V451000  False       False   
4             4   Ford Motor Company           22V825000  False       False   
..          ...                  ...                 ...    ...         ...   
700         700  General Motors, LLC           22V392000  False       False   
701         701  General Motors, LLC           22V427000  False       False   
702         702  General Motors, LLC           22V427000  False       False   
703         703  General Motors, LLC           22V446000  False       False   
704         704  General Motors, LLC           22V446000  False       False   

    ReportReceivedDat

In [3]:
df2.head
#test

<bound method NDFrame.head of      Unnamed: 0         Manufacturer NHTSACampaignNumber parkIt parkOutSide  \
0             0   Ford Motor Company           22V411000  False       False   
1             1   Ford Motor Company           22V451000  False       False   
2             2   Ford Motor Company           22V825000  False       False   
3             3   Ford Motor Company           22V928000  False       False   
4             4   Ford Motor Company           23V070000  False       False   
..          ...                  ...                 ...    ...         ...   
700         700  General Motors, LLC           22V427000  False       False   
701         701  General Motors, LLC           22V446000  False       False   
702         702  General Motors, LLC           22V392000  False       False   
703         703  General Motors, LLC           22V427000  False       False   
704         704  General Motors, LLC           22V446000  False       False   

    ReportReceivedDat

In [5]:

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("datahub.transportation.gov", 'iS5laersRc93IyBG0E5hMBsG9')

# Example authenticated client (needed for non-public datasets):
# client = Socrata(datahub.transportation.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("6axg-epim", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,report_received_date,nhtsa_id,recall_link,manufacturer,subject,component,mfr_campaign_number,recall_type,potentially_affected,defect_summary,consequence_summary,corrective_action,fire_risk_when_parked,do_not_drive,completion_rate
0,2023-12-12T00:00:00.000,23V838000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,"Tesla, Inc.",Autopilot Controls Insufficient to Prevent Misuse,ELECTRICAL SYSTEM,SB-23-00-008,Vehicle,2031220,"Tesla, Inc. (Tesla) is recalling 2012-2023 Mod...",In certain circumstances when Autosteer is eng...,Tesla will release an over-the-air (OTA) softw...,No,No,NaN
1,2023-12-12T00:00:00.000,23V839000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,Great Dane Trailers,Loose Axle Spindle Nut May Cause Wheel Detachment,SUSPENSION,NR (Not Reported),Vehicle,346,Great Dane Trailers (Great Dane) is recalling ...,A loose axle spindle nut can allow the wheel t...,Great Dane will inspect and tighten the axle s...,No,No,NaN
2,2023-12-12T00:00:00.000,23V837000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,Mazda North American Operations,Replacement Air Bag Inflator Incorrectly Insta...,AIR BAGS,6423L,Vehicle,12741,Mazda North American Operations (Mazda) is rec...,An incorrectly installed inflator may not prop...,Dealers will inspect and reinstall the front p...,No,No,NaN
3,2023-12-08T00:00:00.000,23V834000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,"Forest River, Inc.",Insuffeciently Reflective Taillights/FMVSS 108,EXTERIOR LIGHTING,79-1711,Vehicle,30,"Forest River, Inc. (Forest River) is recalling...",Taillights that do not reflect enough light re...,"Dealers will install reflector stickers, free ...",No,No,NaN
4,2023-12-08T00:00:00.000,23V836000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,Harley-Davidson Motor Company,Motorcycle Difficult to Control at High Speeds,ENGINE AND ENGINE COOLING,0185,Vehicle,1464,Harley-Davidson Motor Company (Harley-Davidson...,A motorcycle that is difficult to control can ...,Dealers will update the engine control module ...,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2021-12-17T00:00:00.000,21V989000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,"Mercedes-Benz USA, LLC",Power Steering Assist May Fail,STEERING,2021120016,Vehicle,1,"Mercedes-Benz USA, LLC (MBUSA) is recalling on...",A sudden loss of power steering assist can inc...,A dealer will inspect and repair the power sup...,No,No,100
1996,2021-12-17T00:00:00.000,21V990000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,"Mercedes-Benz USA, LLC",Front Axle Carrier May Corrode,STEERING,2022070009,Vehicle,38312,"Mercedes-Benz USA, LLC. (MBUSA) is recalling c...",Front axle carrier failure may result in a los...,Dealers will inspect and replace the integral ...,No,No,85.27
1997,2021-12-17T00:00:00.000,21V991000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,"Corp. Micro Bird, Inc.",Emergency Exit Release Handle Obstructed/FMVSS...,EQUIPMENT,21-089-FUC,Vehicle,75,"Corp. Micro Bird, Inc. (Micro Bird) is recalli...",Occupants may have to unfold the seat to acces...,Micro Bird will provide inspection instruction...,No,No,13.33
1998,2021-12-17T00:00:00.000,21V988000,{'url': 'https://www.nhtsa.gov/recalls?nhtsaId...,"Forest River, Inc.",Carbon Monoxide May Enter Cabin,EQUIPMENT,15-1444,Vehicle,111,"Forest River, Inc. (Forest River) is recalling...",Carbon monoxide inside the vehicle can increas...,Dealers will inspect the generator compartment...,No,No,36.94
